In [1]:
import tensorflow as tf
import tensorflow_hub as hub


In [2]:
image_path = '/Users/williamstone/main_dir/school/oswego/capstone/misc/MoveNet-Judo-Model-Training/training-app/vids/frames/frame_0000.jpg'

image = tf.io.read_file(image_path)
image = tf.compat.v1.image.decode_jpeg(image)

image = tf.expand_dims(image, axis=0)
# Resize and pad

image = tf.cast(tf.image.resize_with_pad(image, 256, 256), dtype=tf.int32)


In [3]:
# Download the model
model = hub.load("https://www.kaggle.com/models/google/movenet/TensorFlow2/multipose-lightning/1")

movenet = model.signatures["serving_default"]


In [4]:
outputs = movenet(image)
keypoints = outputs['output_0']

print(outputs)
print(keypoints)

{'output_0': <tf.Tensor: shape=(1, 6, 56), dtype=float32, numpy=
array([[[3.42298478e-01, 5.23988783e-01, 5.10326564e-01, 3.23219955e-01,
         5.26891232e-01, 4.37180966e-01, 3.22641730e-01, 5.25686800e-01,
         4.64417070e-01, 3.14971626e-01, 5.51533401e-01, 6.42783403e-01,
         3.16139519e-01, 5.57659328e-01, 6.61030114e-01, 3.85317773e-01,
         5.90250194e-01, 5.44890821e-01, 3.89642477e-01, 5.72423697e-01,
         5.14120698e-01, 5.21609485e-01, 6.02508962e-01, 4.26445067e-01,
         5.02427101e-01, 5.94263375e-01, 2.67259568e-01, 5.27397215e-01,
         5.05942881e-01, 4.80410755e-01, 5.19883275e-01, 5.18342137e-01,
         2.40289226e-01, 6.33649349e-01, 6.15526438e-01, 6.92716420e-01,
         6.31168067e-01, 5.92298865e-01, 6.12613738e-01, 7.72947311e-01,
         5.95932186e-01, 2.37743706e-01, 7.66473055e-01, 5.52031755e-01,
         3.80552649e-01, 7.51837432e-01, 6.28916562e-01, 4.94657680e-02,
         7.35446811e-01, 5.99892139e-01, 6.65213913e-02, 2.

In [5]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [6]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [7]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)
    

In [8]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

In [9]:
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

In [11]:
cap = cv2.VideoCapture('/Users/williamstone/main_dir/school/oswego/capstone/misc/MoveNet-Judo-Model-Training/training-app/vids/vid2.mov')
while cap.isOpened():
    ret, frame = cap.read()
    
    # Resize the image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384, 640)
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    # Render keypoints
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.1)
    
    cv2.imshow('MoveNet Multipose', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
    

AttributeError: 'NoneType' object has no attribute 'copy'